In [1]:
# Load the model

In [21]:
pip install bert_score

Note: you may need to restart the kernel to use updated packages.


In [52]:
import os
import torch
from transformers import AutoModelForCausalLM
from datasets import load_dataset
from bert_score import score
import time
from transformers import AutoTokenizer
import numpy as np


In [69]:
model_path = "/mnt/object/my_model.pth"

In [16]:
model_size = os.path.getsize(model_path) 
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")


Model Size on Disk: 31415.38 MB


In [5]:
print(torch.cuda.is_available())

True


In [19]:
dataset = load_dataset("neulab/conala", split="train[:2]", trust_remote_code=True)  # Just a few samples for quick test

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

conala.py:   0%|          | 0.00/4.30k [00:00<?, ?B/s]

conala-paired-train.json:   0%|          | 0.00/518k [00:00<?, ?B/s]

conala-paired-test.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
model = torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu")

/tmp/ipykernel_563/3351852161.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu")

OrderedDict([('model.embed_tokens.weight',
              tensor([[ 2.0410e-01, -1.1902e-02, -2.9541e-02,  ..., -3.5400e-02,
                        7.3730e-02, -1.0596e-01],
                      [-3.8477e-01,  2.8809e-02, -1.9287e-02,  ..., -1.0071e-02,
                        7.7637e-02, -6.6406e-02],
                      [-1.3477e-01, -3.8086e-02, -2.6489e-02,  ..., -1.2146e-02,
                       -4.0588e-03,  2.7924e-03],
                      ...,
                      [-2.8992e-03, -9.9945e-04,  1.9836e-03,  ..., -1.9073e-04,
                        6.2256e-03,  1.8768e-03],
                      [-1.3885e-03, -4.5776e-03, -8.3008e-03,  ...,  9.3384e-03,
                        7.9346e-04, -1.6403e-03],
                      [-3.8910e-03,  8.3008e-03,  3.8338e-04,  ..., -2.2888e-03,
                        6.7749e-03,  1.1841e-02]], device='cuda:0', dtype=torch.float16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0503,  0.0027,  0.038

In [13]:
model_address = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"

In [15]:
model = AutoModelForCausalLM.from_pretrained(model_address)
model.load_state_dict(torch.load(model_path, map_location="cuda"))
model.eval()


/tmp/ipykernel_563/2244306946.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location="cuda"))


DeepseekV2ForCausalLM(
  (model): DeepseekV2Model(
    (embed_tokens): Embedding(102400, 2048)
    (layers): ModuleList(
      (0): DeepseekV2DecoderLayer(
        (self_attn): DeepseekV2Attention(
          (q_proj): Linear(in_features=2048, out_features=3072, bias=False)
          (kv_a_proj_with_mqa): Linear(in_features=2048, out_features=576, bias=False)
          (kv_a_layernorm): DeepseekV2RMSNorm()
          (kv_b_proj): Linear(in_features=512, out_features=4096, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): DeepseekV2YarnRotaryEmbedding()
        )
        (mlp): DeepseekV2MLP(
          (gate_proj): Linear(in_features=2048, out_features=10944, bias=False)
          (up_proj): Linear(in_features=2048, out_features=10944, bias=False)
          (down_proj): Linear(in_features=10944, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekV2RMSNorm()
        (post_atten

In [38]:
tokenizer = AutoTokenizer.from_pretrained(
    model_address,
    trust_remote_code=True,  # Needed if the tokenizer uses custom logic
    use_fast=False           # More compatible for DeepSeek models
)

In [50]:
def generate_code_with_profiling(model, tokenizer, prompt, max_new_tokens=128):
    """
    Generates a code review comment using the given model and tokenizer,
    and reports inference time and memory usage.
    Handles both CPU and large GPU models with memory safety.
    """
    device = next(model.parameters()).device  # Get device from loaded model

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    print(f" Prompt token length: {inputs['input_ids'].shape[1]}")

    if device.type == "cuda":
        print(" CUDA is available, measuring GPU memory usage...")
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        memory_before = torch.cuda.memory_allocated(device)
    else:
        print(" Running on CPU")
        memory_before = 0

    start_time = time.time()

    try:
        with torch.no_grad():
            print(" Generating...")
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                use_cache=True
            )
            print(" Generation complete.")
    except Exception as e:
        print(f" Generation failed: {e}")
        return "", 0.0, 0

    if device.type == "cuda":
        torch.cuda.synchronize()
        memory_after = torch.cuda.memory_allocated(device)
        memory_usage = memory_after - memory_before
    else:
        memory_usage = 0

    end_time = time.time()
    inference_time = end_time - start_time
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return output_text, inference_time, memory_usage


In [51]:
# Your GitHub-style input
diff_hunk = "@@ -1518,7 +1518,10 @@ private File createWindowsSshAskpass(SSHUserPrivateKey sshUser) throws IOExcepti\n         PrintWriter w = null;\n         try {\n             w = new PrintWriter(ssh);\n-            w.println(\"echo \\\"\" + Secret.toString(sshUser.getPassphrase()) + \"\\\"\");\n+            // avoid echoing command as part of the password\n+            w.println(\"@echo off\");\n+            // no need for quotes on windows echo -- they will get echoed too\n+            w.println(\"echo \" + Secret.toString(sshUser.getPassphrase()));"

human_comment = """This code only seems to be invoked when a passphrase has been assigned to the private key... (your full comment here)"""  # Truncated for brevity, use full body

# Prompt for code review comment generation
prompt = f"### Instruction:\nAnalyze the following code diff and provide a code review comment:\n{diff_hunk}\n\n### Response:"

# Generate model output (comment)
generated_comment, inference_time, memory_usage = generate_code_with_profiling(model, tokenizer, prompt)

# Evaluate semantic similarity
P, R, F1 = score([generated_comment], [human_comment], lang="en", device="cpu", verbose=False)
bert_f1 = F1[0].item()

# Print results
print("\nHuman Review Comment:\n", human_comment)
print("\n Model-Generated Comment:\n", generated_comment)
print(f"\n BERTScore F1: {bert_f1:.4f}")
print(f" Inference time: {inference_time:.2f}s  |  Memory used: {memory_usage / 1e6:.2f} MB")


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


 Prompt token length: 166
 CUDA is available, measuring GPU memory usage...
 Generating...
 Generation failed: 'DynamicCache' object has no attribute 'get_max_length'


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Human Review Comment:
 This code only seems to be invoked when a passphrase has been assigned to the private key... (your full comment here)

 Model-Generated Comment:
 

 BERTScore F1: 0.0000
 Inference time: 0.00s  |  Memory used: 0.00 MB


In [35]:
import torch
print(torch.cuda.is_available())  # Should return True for GPU support


True


In [54]:
def measure_latency(model, tokenizer, prompt, num_trials=100, max_new_tokens=64):
    """
    Measures inference latency for a single sample across multiple trials.
    """
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Warm-up
    with torch.no_grad():
        _ = model.generate(**inputs, max_new_tokens=max_new_tokens)

    latencies = []
    with torch.no_grad():
        for _ in range(num_trials):
            start_time = time.time()
            _ = model.generate(**inputs, max_new_tokens=max_new_tokens)
            latencies.append(time.time() - start_time)

    print(f"\n Inference Latency Benchmark (n={num_trials}):")
    print(f" - Median Latency: {np.percentile(latencies, 50) * 1000:.2f} ms")
    print(f" - 95th Percentile: {np.percentile(latencies, 95) * 1000:.2f} ms")
    print(f" - 99th Percentile: {np.percentile(latencies, 99) * 1000:.2f} ms")
    print(f" - Inference Throughput: {num_trials / np.sum(latencies):.2f} samples/sec")

    return latencies


In [59]:
from types import MethodType

def patched_prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, inputs_embeds=None, **kwargs):
    if past_key_values is not None:
        if hasattr(past_key_values, "get_seq_length"):
            cache_length = past_key_values.get_seq_length()
            past_length = past_key_values.seen_tokens
            max_cache_length = None  # <<< Prevent crashing
        else:
            cache_length = past_length = past_key_values[0][0].shape[2]
            max_cache_length = None
    else:
        cache_length = past_length = 0
        max_cache_length = None

    if attention_mask is not None and past_key_values is not None:
        if attention_mask.shape[1] > input_ids.shape[1]:
            input_ids = input_ids[:, -(attention_mask.shape[1] - past_length):]
        elif past_length < input_ids.shape[1]:
            input_ids = input_ids[:, past_length:]

    position_ids = kwargs.get("position_ids", None)
    if attention_mask is not None and position_ids is None:
        position_ids = attention_mask.long().cumsum(-1) - 1
        position_ids.masked_fill_(attention_mask == 0, 1)
        if past_key_values:
            position_ids = position_ids[:, -input_ids.shape[1]:]

    if inputs_embeds is not None and past_key_values is None:
        model_inputs = {"inputs_embeds": inputs_embeds}
    else:
        model_inputs = {"input_ids": input_ids}

    model_inputs.update({
        "position_ids": position_ids,
        "past_key_values": past_key_values,
        "use_cache": kwargs.get("use_cache"),
        "attention_mask": attention_mask,
    })
    return model_inputs

# Patch DeepSeek model
model.prepare_inputs_for_generation = MethodType(patched_prepare_inputs_for_generation, model)


In [55]:
def measure_batch_throughput(model, tokenizer, prompt, batch_size=4, num_batches=50, max_new_tokens=64):
    """
    Measures batch throughput for a model using repeated trials.
    """
    device = next(model.parameters()).device
    batch_prompts = [prompt] * batch_size
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Warm-up
    with torch.no_grad():
        _ = model.generate(**inputs, max_new_tokens=max_new_tokens)

    batch_times = []
    with torch.no_grad():
        for _ in range(num_batches):
            start_time = time.time()
            _ = model.generate(**inputs, max_new_tokens=max_new_tokens)
            batch_times.append(time.time() - start_time)

    total_samples = batch_size * num_batches
    throughput = total_samples / np.sum(batch_times)

    print(f"\n Batch Throughput Benchmark (batch={batch_size}, n={num_batches}):")
    print(f" - Throughput: {throughput:.2f} samples/sec")

    return throughput


In [60]:
prompt = "### Instruction:\nGenerate a Python function to calculate Fibonacci numbers.\n\n### Response:"

# Measure single-sample latency
measure_latency(model, tokenizer, prompt, num_trials=100)

# Measure batch throughput
# measure_batch_throughput(model, tokenizer, prompt, batch_size=8, num_batches=50)


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`

In [62]:
model.compile()

In [63]:
import os
import time
import numpy as np
import torch
import onnx
import onnxruntime as ort
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [65]:
device= torch.device("cuda")

In [67]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.eval()

dummy_text = "### Instruction: Review the following code:\ndef add(a, b): return a + b\n\n### Response:"
dummy_inputs = tokenizer(dummy_text, return_tensors="pt")

torch.onnx.export(
    model,
    args=(dummy_inputs["input_ids"],),
    f="code-review-model.onnx",
    input_names=["input_ids"],
    output_names=["logits"],
    dynamic_axes={"input_ids": {0: "batch_size", 1: "sequence_length"}, "logits": {0: "batch_size"}},
    opset_version=17
)

print("Exported ONNX model for GitHub code review bot.")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/transformers/modeling_attn_mask_utils.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/opt/conda/lib/python3.12/site-packages/transformers/modeling_attn_mask_utils.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/home/jovyan/.cache/huggingface/modules/transformers_modules/deepseek-ai/DeepSeek-Coder-V2-Lite-Base/ea9b066cee82f82906fdd58898cb3788b1c5d770/modeling_deepseek.py:875: TracerWarning: Converting a tensor to

Exported ONNX model for GitHub code review bot.


In [68]:
onnx_model_size = os.path.getsize(onnx_model_path) 


NameError: name 'onnx_model_path' is not defined